## Helper script
This is a helper sript defined to find the DOY of each plume flyby to match the Cassini database documentation.

In [1]:
import pandas as pd
import re
from pathlib import Path
import pandas as pd

In [2]:
# Get the DOY of all flybys to match the CDA databases to download

flybys = [
    ("E1",  "2005-03-09 09:08:00"),
    ("E2",  "2005-07-14 19:55:14"),
    ("E3",  "2008-03-12 19:06:12"),
    ("E4",  "2008-08-11 22:21:21"),
    ("E5",  "2008-10-09 17:14:51"),
    ("E6",  "2008-10-31 17:14:51"),
    ("E7",  "2009-11-02 19:41:57"),
    ("E9",  "2010-04-28 19:09:23"),
    ("E10", "2010-05-18 19:10:00"),
    ("E12", "2010-11-30 12:06:00"),
    ("E13", "2010-12-21 13:11:00"),
    ("E14", "2011-10-01 18:40:00"),
    ("E15", "2011-10-19 19:25:00"),
    ("E16", "2011-11-06 20:03:00"),
    ("E17", "2012-03-27 18:30:00"),
    ("E18", "2012-04-14 18:08:00"),
    ("E19", "2012-05-02 18:12:00"),
    ("E20", "2015-10-14 05:22:00"),
    ("E21", "2015-10-28 15:22:00"),
]

for name, date_str in flybys:
    dt = pd.to_datetime(date_str)
    doy = dt.dayofyear
    print(f"{name}: Year: {dt.year} | DOY: {doy}")

E1: Year: 2005 | DOY: 68
E2: Year: 2005 | DOY: 195
E3: Year: 2008 | DOY: 72
E4: Year: 2008 | DOY: 224
E5: Year: 2008 | DOY: 283
E6: Year: 2008 | DOY: 305
E7: Year: 2009 | DOY: 306
E9: Year: 2010 | DOY: 118
E10: Year: 2010 | DOY: 138
E12: Year: 2010 | DOY: 334
E13: Year: 2010 | DOY: 355
E14: Year: 2011 | DOY: 274
E15: Year: 2011 | DOY: 292
E16: Year: 2011 | DOY: 310
E17: Year: 2012 | DOY: 87
E18: Year: 2012 | DOY: 105
E19: Year: 2012 | DOY: 123
E20: Year: 2015 | DOY: 287
E21: Year: 2015 | DOY: 301


In [3]:
# check if we downloaded the correct databases

flybys = [(n, pd.Timestamp(t)) for n, t in flybys]

root = Path("../Dataset")
vol_pattern = re.compile(r"COCDA_\d{4}")
volumes = [d for d in root.iterdir() if d.is_dir() and vol_pattern.fullmatch(d.name)]

for vol in sorted(volumes):
    event_files = sorted(vol.glob("DATA/**/CDAEVENTS_*.TAB"))

    min_t, max_t = None, None
    for f in event_files:
        try:
            # Read the time
            df = pd.read_fwf(f, colspecs=[(11, 28)], names=["EVENT_TIME"], dtype=str)
            df["EVENT_TIME"] = pd.to_datetime(df["EVENT_TIME"], errors="coerce", format="%Y-%jT%H:%M:%S")
            df = df.dropna(subset=["EVENT_TIME"])

            # determine min and maxima
            local_min = df["EVENT_TIME"].min()
            local_max = df["EVENT_TIME"].max()
            if not min_t or local_min < min_t:
                min_t = local_min
            if not max_t or local_max > max_t:
                max_t = local_max

        except Exception as e:
            print(f"[ERROR] Failed parsing {f.name}: {e}")

    if not min_t or not max_t:
        print(f"[INFO] {vol.name}: Could not determine time range")
        continue

    print(f"\n{vol.name}: {min_t:%Y-%m-%d %H:%M:%S} -> {max_t:%Y-%m-%d %H:%M:%S}")

    # Check which flybys are in this range
    inside = [f"{n} ({t:%Y-%m-%d %H:%M:%S})" for n, t in flybys if min_t <= t <= max_t]
    if inside:
        print(f"  Flybys inside {vol.name}: {', '.join(inside)}")
    else:
        print(f"  No flybys inside {vol.name}.")


COCDA_0007: 2005-01-01 00:02:42 -> 2005-03-31 23:59:49
  Flybys inside COCDA_0007: E1 (2005-03-09 09:08:00)

COCDA_0012: 2005-07-01 00:00:44 -> 2005-07-21 23:58:48
  Flybys inside COCDA_0012: E2 (2005-07-14 19:55:14)

COCDA_0045: 2008-03-02 01:01:41 -> 2008-03-31 23:57:25
  Flybys inside COCDA_0045: E3 (2008-03-12 19:06:12)

COCDA_0050: 2008-08-04 13:36:30 -> 2008-08-26 13:39:19
  Flybys inside COCDA_0050: E4 (2008-08-11 22:21:21)

COCDA_0052: 2008-10-01 02:00:50 -> 2008-11-01 00:01:49
  Flybys inside COCDA_0052: E5 (2008-10-09 17:14:51), E6 (2008-10-31 17:14:51)

COCDA_0057: 2009-10-01 00:12:56 -> 2009-12-31 23:54:25
  Flybys inside COCDA_0057: E7 (2009-11-02 19:41:57)

COCDA_0059: 2010-04-01 00:35:26 -> 2010-06-30 23:35:06
  Flybys inside COCDA_0059: E9 (2010-04-28 19:09:23), E10 (2010-05-18 19:10:00)

COCDA_0061: 2010-10-01 03:41:14 -> 2010-12-31 22:29:26
  Flybys inside COCDA_0061: E12 (2010-11-30 12:06:00), E13 (2010-12-21 13:11:00)

COCDA_0069: 2011-10-01 01:35:37 -> 2011-10-31 